In [ ]:
!pip install -r requirements.txt

In [ ]:
import sys
print(sys.executable)

In [1]:
# %%
import json
import os
import shutil
import src.constants as con


# %%
with open(os.getcwd()+ con.API_KEYS_PATH, "r") as f:
    credentials = json.load(f)


os.environ['LANGCHAIN_TRACING_V2'] = credentials["LANGCHAIN_TRACING_V2"]
os.environ['LANGCHAIN_ENDPOINT'] = credentials["LANGCHAIN_ENDPOINT"]
os.environ['LANGCHAIN_API_KEY'] = credentials["LANGCHAIN_API_KEY"]
os.environ['OPENAI_API_KEY'] = credentials["OPENAI_API_KEY"]
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials["GOOGLE_APPLICATION_CREDENTIALS"]


from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
import streamlit as st 


from src.loader import load_google_docs
import src.prompts as pr
from src.utils import token_generator, format_docs
from src.vectorstore import create_vectorstore_from_documents, split_documents, add_documents_to_vectorstore


/Users/agostinocalamia/repositories/jobportal_chatbot/src/vectorstore.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  def create_vectorstore_from_documents(documents, vectorstore=Chroma(), embedding=OpenAIEmbeddings()):
2024-09-15 19:56:41.786 INFO    chromadb.telemetry.product.posthog: Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [ ]:
os.getcwd()

In [3]:

# %%
if not os.path.exists('.credentials/token.json'):
    token_generator()

    shutil.move(os.getcwd() + '/token.json', os.path.join(os.getcwd() + '/.credentials/', os.path.basename('token.json')))

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=314599650974-clcign9rquanpbh5sl5fqf2nrf74gj41.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A54750%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=5WVA7tPeMdX5bHaR1LCQKFFKkAInDM&access_type=offline


2024-09-15 19:58:30.027 INFO    google_auth_oauthlib.flow: "GET /?state=5WVA7tPeMdX5bHaR1LCQKFFKkAInDM&code=4/0AQlEd8y-6gORh8g3CU9IIQyNNiJ_XvbblpTbHJG12pXmBhcNYAIOSo4bZRMsa0XXJBXNFQ&scope=https://www.googleapis.com/auth/drive HTTP/1.1" 200 65


In [5]:


# Define the document loader
text_splitter = RecursiveCharacterTextSplitter(chunk_size=con.SPLITTER_CHUNK_SIZE, chunk_overlap=con.SPLITTER_CHUNK_OVERLAP)
# Define the LLM
llm = ChatOpenAI(model_name=con.MODEL_NAME, temperature=con.MODEL_TEMPERATURE)

# %%
#### ADDING INTERVIEW CONTEXT ####
print('load interview coneent')

job_docs = load_google_docs(folder_id = con.JOB_FOLDER_ID, file_types=["document", "pdf"])
job_splits = split_documents(job_docs, text_splitter=text_splitter)
vector_db = create_vectorstore_from_documents(job_splits)

load interview coneent


2024-09-15 19:58:55.621 INFO    httpx: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [ ]:



# %%

retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 5})

if con.PROMPT_TEMPLATE == "basic":
    prompt = ChatPromptTemplate.from_template(pr.create_basic_prompt())
else:
    prompt = hub.pull("rlm/rag-prompt")

# Chain
rag_chain = (
     {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
